In [4]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# -------------------------
#   To-do
# -------------------------
# 1. add logger
# 2. make image drawer overlay mask on image
# 3. make iterable
# 4. make verbose turn on and off
# 5. write pydoc

# tensorflow Module
import tensorflow as tf
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import losses

import segmentation_models as sm

# python basic Module
import os
import sys
import types
import progressbar
from datetime import datetime
from shutil import copy
from pickle import dump, load

# math, image, plot Module
import numpy as np
import cv2
import matplotlib.pyplot as plt  # TBD

# email Module
import smtplib
from email.mime.text import MIMEText
from email.header import Header

from data_loader.medical_segmentation_data_loader_v1 import DataLoader

from gan_module.model import build_generator, build_discriminator
from gan_module.draw_images import ImageDrawer
from gan_module.custom_loss import f1_loss_for_training, f1_score, dice_loss_for_training

# set GPU memory growth allocation
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

"""
if you don't have nvidia-gpu, try plaidml! but it will works tensorflow 1.x.x
# pip install -U plaidml-keras
# plaidml-setup
"""
# use_plaidml = False
# if use_plaidml :
#     import plaidml.keras
#     plaidml.keras.install_backend()
#     os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

class Pix2PixSegmentation:
    def __init__(
        self,
        generator_power=32,
        discriminator_power=32,
        generator_learning_rate=1e-4,
        discriminator_learning_rate=1e-4,
        find_error=False,
        temp_weights_path=".",
        draw_images=True,
        on_memory=True,
        test=False
    ):
        # smtp info
        self.smtp_host = "smtp.gmail.com"
        self.smtp_port = 465
        self.smtp_id = "rpa.manager0001@gmail.com"
        self.smtp_password = "!rpa.admin!23"
        self.smtp_to_addr = "tobeor3009@gmail.com"

        # Input shape
        self.img_rows = 512
        self.img_cols = 512
        self.input_channels = 3
        self.output_channels = 1
        self.input_img_shape = (
            self.img_rows, self.img_cols, self.input_channels)
        self.output_img_shape = (
            self.img_rows, self.img_cols, self.output_channels)
        # set parameter
        self.start_epoch = None
        self.history = {"generator_loss": [], "f1_score_train": [], "f1_score_valid": []}
        self.f1_loss_ratio = 16
        self.huber_loss_ratio = 64
        self.find_error = find_error
        self.find_error_epoch = 30
        self.error_list = []
        self.temp_weights_path = temp_weights_path

        # Configure data loader
        self.dataset_name = "tumor"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            img_res=(self.img_rows, self.img_cols),
            on_memory=on_memory, test=test
        )
        self.train_loaded_data, self.valid_loaded_data = self.data_loader.load_all()
        if test:
            self.train_loaded_data_len = 20
            self.valid_loaded_data_len = 20
        else:
            self.train_loaded_data_len = self.data_loader.train_data_length
            self.valid_loaded_data_len = self.data_loader.valid_data_length

        self.train_loaded_data_index = np.arange(self.train_loaded_data_len)
        self.valid_loaded_data_index = np.arange(self.valid_loaded_data_len)

        # Configure Image Drawer
        self.draw_images = draw_images
        self.image_drawer = ImageDrawer(
            dataset_name=self.dataset_name, data_loader=self.data_loader
        )
        # training parameters
        self.learning_schedule = [
            50,
            100,
            150,
            200,
            250,
            300,
            350,
            400,
            450,
            500,
            550
        ]
        self.discriminator_acc_previous = 0.5
        self.discriminator_acces = np.array(
            [0.5 for _ in range(self.train_loaded_data_len)])
        self.discriminator_acces_previous = self.discriminator_acces.copy()
        self.generator_losses = np.array(
            [1 for _ in range(self.train_loaded_data_len)], dtype=np.float32)
        self.generator_losses_previous = self.generator_losses.copy()
        self.generator_loss_min = 100
        self.generator_loss_previous = 100
        self.generator_loss_max_previous = 1000
        self.generator_loss_max_min = 1000
        self.generator_loss_min_min = 1000
        self.weight_save_stack = False
        self.training_end_stack = 0
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 2)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.generator_power = generator_power
        self.discriminator_power = discriminator_power
        self.generator_learning_rate = generator_learning_rate
        self.discriminator_learning_rate = discriminator_learning_rate
        generator_optimizer = Nadam(self.generator_learning_rate)
        discriminator_optimizer = Nadam(self.discriminator_learning_rate)

        # layer Component
        self.kernel_initializer = RandomNormal(mean=0.0, stddev=0.02)

        # Build the generator
        self.generator = build_generator(
            input_img_shape=self.input_img_shape,
            output_channels=self.output_channels,
            generator_power=self.generator_power,
            kernel_initializer=self.kernel_initializer,
        )
        self.generator.compile(
            loss=tf.keras.losses.Huber(),
            optimizer=generator_optimizer,
            metrics=["accuracy"],
        )

        # Build and compile the discriminator
        self.discriminator = build_discriminator(
            input_img_shape=self.input_img_shape,
            output_img_shape=self.output_img_shape,
            discriminator_power=self.discriminator_power,
            kernel_initializer=self.kernel_initializer,
        )
        # self.discriminator = self.build_discriminator()
        # 'mse' or tf.keras.losses.Huber() tf.keras.losses.LogCosh()
        self.discriminator.compile(
            loss=tf.keras.losses.LogCosh(),
            optimizer=discriminator_optimizer,
            metrics=["accuracy"],
        )

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Input images and their conditioning images
        original_img = Input(shape=self.input_img_shape)
        masked_img = Input(shape=self.output_img_shape)
        # generate image from original_img for target masked_img
        model_masked_img = self.generator(original_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        # Discriminators determines validity of translated images / condition pairs
        model_validity = self.discriminator([original_img, model_masked_img])
        # give score by
        # 1. how generator trick discriminator
        # 2. how generator's image same as real photo in pixel
        # 3. if you want change loss, see doc https://keras.io/api/losses/
        # 4. 'mse', 'mae', tf.keras.losses.LogCosh(),  tf.keras.losses.Huber()
        self.combined = Model(
            inputs=[original_img, masked_img],
            outputs=[model_validity, model_masked_img, model_masked_img],
        )
        self.combined.compile(
            loss=[
                tf.keras.losses.LogCosh(),
                sm.losses.BinaryFocalLoss(),
                dice_loss_for_training
            ],
            loss_weights=[2, self.huber_loss_ratio, self.f1_loss_ratio],
            optimizer=generator_optimizer
        )

    def train(self, epochs, batch_size=1, sample_interval=50, epoch_shuffle_term=10):

        start_time = datetime.now()
        
        # Adversarial loss ground truths
        self.training_end_stack = 0
        self.batch_size = batch_size
        valid_patch = np.ones((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        fake_patch = np.zeros((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        if self.start_epoch is None:
            self.start_epoch = 0
        for epoch in range(self.start_epoch, epochs):
            bar = progressbar.ProgressBar(maxval=self.train_loaded_data_len).start()            
            batch_i = 0
            
            discriminator_losses = []
            generator_loss_max_in_epoch = 0
            generator_loss_min_in_epoch = 1000
            generator_current_learning_rate = self.learning_rate_scheduler(
                self.generator_learning_rate,
                epoch,
            )
            discriminator_current_learning_rate = self.learning_rate_scheduler(
                self.discriminator_learning_rate,
                epoch,
            )
            ratio = 0.25
            
            generator_loss_median = np.quantile(self.generator_losses_previous, ratio)
            print(f"current_generator_loss_median : {generator_loss_median}")
                       
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            keras_backend.set_value(
                self.generator.optimizer.learning_rate, generator_current_learning_rate * 2
            )
            keras_backend.set_value(
                self.combined.optimizer.learning_rate, generator_current_learning_rate
            )
            # shffle data maybe
            if epoch % epoch_shuffle_term == 0 and not isinstance(self.train_loaded_data, types.GeneratorType):
                np.random.shuffle(self.train_loaded_data_index)
                
            if (self.discriminator_acc_previous < 0.8 and epoch % epoch_shuffle_term) or self.discriminator_acc_previous < 0.6:
                discriminator_learning = True
                print("discriminator_learning is True")
            else:
                discriminator_learning = False
                print("discriminator_learning is False")
            
            
            while batch_i + self.batch_size <= self.train_loaded_data_len:
                bar.update(batch_i)
                
                batch_index = self.train_loaded_data_index[batch_i: batch_i +
                                                         self.batch_size]
                
                original_img = self.train_loaded_data[0][batch_index]
                masked_img = self.train_loaded_data[1][batch_index]
                
                model_masked_img = self.generator.predict_on_batch(
                    original_img)

                # forTest
                self.masked_img = masked_img
                self.original_img = original_img
                self.model_masked_img = model_masked_img
                self.valid_path = valid_patch
                self.fake_patch = fake_patch
                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Train Discriminator for valid image if it failed to detect fake image
                generator_loss_previous = np.mean(
                    self.generator_losses_previous[batch_index])
                
                if discriminator_learning:
                    self.discriminator.train_on_batch(
                        [original_img, masked_img], valid_patch)
                    
                batch_discriminator_acc_previous = np.mean(
                    self.discriminator_acces_previous[batch_index])
                
                # Train generator for image detected by discriminator
                if (batch_discriminator_acc_previous > 0.5 and 
                    epoch > 1 and 
                    epoch < 20 and
                    generator_loss_previous > generator_loss_median) :
                    self.generator.train_on_batch(
                        original_img, masked_img)
                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators

                generator_loss = self.combined.train_on_batch(
                    [original_img, masked_img],
                    [valid_patch, masked_img, masked_img],
                )
                # train discriminator for fake image if it failed to detect fake image
                if batch_discriminator_acc_previous <= 0.5 or epoch == 0:
                    discriminator_loss = self.discriminator.train_on_batch(
                        [original_img, model_masked_img], fake_patch)
                else:
                    discriminator_loss = self.discriminator.test_on_batch(
                        [original_img, model_masked_img], fake_patch)

                self.discriminator_acces[batch_index] = discriminator_loss[1]
                self.generator_losses[batch_index] = generator_loss[0]
                elapsed_time = datetime.now() - start_time
                if batch_i % sample_interval == 0:
                    # Plot the progress
                    print(
                        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s"
                        % (
                            epoch,
                            epochs,
                            batch_i,
                            self.train_loaded_data_len,
                            discriminator_loss[0],
                            100 * discriminator_loss[1],
                            generator_loss[0],
                            elapsed_time,
                        )
                    )

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0 and self.draw_images:
                    self.image_drawer.sample_images(
                        self.generator, epoch, batch_i)

                discriminator_losses.append(discriminator_loss[0])
                # loss 가 가장 높은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss[0] > generator_loss_max_in_epoch:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=True,
                        )
                    generator_loss_max_in_epoch = generator_loss[0]
                # loss 가 가장 낮은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss_min_in_epoch > generator_loss[0]:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=False,
                        )
                    generator_loss_min_in_epoch = generator_loss[0]

                # 한 배치 끝
                batch_i += self.batch_size
            # training batch 사이클 끝
            print(f"discriminator_acces : {np.mean(self.discriminator_acces)}")
            print(
                f"Mean generator_loss : {np.mean(self.generator_losses)}")
            print(f"Max generator_loss : {np.max(self.generator_losses)}")
            print(f"Min generator_loss : {np.min(self.generator_losses)}")
            print(
                f"generator loss decrease : {self.generator_loss_min - np.mean(self.generator_losses)}"
            )
            print(
                f"generator loss decrease ratio : ({np.mean(self.generator_losses) / self.generator_loss_min})"
            )
            print(
                f"Max generator loss decrease : {self.generator_loss_max_previous - np.max(self.generator_losses)}"
            )
            print(
                f"current lowest generator loss : {self.generator_loss_min}")
            print(
                f"current Learning_rate = {generator_current_learning_rate}")
            # rollback if loss not converge
            if np.mean(self.generator_losses) / self.generator_loss_min < 1.1:
                if self.generator_loss_min > np.mean(self.generator_losses):
                    self.generator_loss_min = np.mean(self.generator_losses)
                    self.generator_loss_max_min = generator_loss_max_in_epoch
                    self.generator_loss_min_min = generator_loss_min_in_epoch
                    self.weight_save_stack = True
                    self.save_study_info()
                    print("save weights")
                #compute f1_score
                train_f1_loss_list = []
                train_f1_score_list = []
                train_predict_mini_batch_size = 1
                for index in range(0, self.train_loaded_data_len, train_predict_mini_batch_size):

                    train_masked_img = self.train_loaded_data[1][index:index +
                                                                 train_predict_mini_batch_size]
                    train_model_masked_img = self.generator.predict_on_batch(
                        self.train_loaded_data[0][index:index + train_predict_mini_batch_size])

                    train_f1_loss = f1_loss_for_training(
                        train_masked_img, np.squeeze(train_model_masked_img))
                    train_f1_score = f1_score(
                        train_masked_img, np.squeeze(train_model_masked_img))
                    train_f1_loss_list.append(train_f1_loss)
                    train_f1_score_list.append(train_f1_score)
                print(
                    f"train_f1_loss : {np.mean(train_f1_loss_list) * self.f1_loss_ratio}")
                print(f"train_f1_score : {1 - np.mean(train_f1_loss_list)}")
                print(f"train_f1_rounded_score : {np.mean(train_f1_score_list)}")

                valid_f1_loss_list = []
                valid_f1_score_list = []
                valid_predict_mini_batch_size = 1
                for index in range(0, self.valid_loaded_data_len, valid_predict_mini_batch_size):

                    valid_masked_img = self.valid_loaded_data[1][index:index +
                                                                 valid_predict_mini_batch_size]
                    valid_model_masked_img = self.generator.predict_on_batch(
                        self.valid_loaded_data[0][index:index + valid_predict_mini_batch_size])

                    valid_f1_loss = f1_loss_for_training(
                        valid_masked_img, np.squeeze(valid_model_masked_img))
                    valid_f1_score = f1_score(
                        valid_masked_img, np.squeeze(valid_model_masked_img))
                    valid_f1_loss_list.append(valid_f1_loss)
                    valid_f1_score_list.append(valid_f1_score)
                print(
                    f"valid_f1_loss : {np.mean(valid_f1_loss_list) * self.f1_loss_ratio}")
                print(f"valid_f1_score : {1 - np.mean(valid_f1_loss_list)}")
                print(f"valid_f1_rounded_score : {np.mean(valid_f1_score_list)}")
            else:
                print("loss decrease. skip compute f1_score")
                self.load_best_weights()

            # previous generator_loss 갱신
            self.generator_loss_previous = np.mean(self.generator_losses)
            self.generator_loss_max_previous = generator_loss_max_in_epoch

            if epoch >= 10 and self.weight_save_stack:
                copy(
                    "generator.h5",
                    "./generator_weights/generator_"
                    + str(round(self.generator_loss_min, 5))
                    + "_"
                    + str(round(self.generator_loss_max_min, 5))
                    + ".h5",
                )
                self.weight_save_stack = False

            self.discriminator_acc_previous = np.mean(self.discriminator_acces)
            self.discriminator_acces_previous = self.discriminator_acces.copy()
            self.generator_losses_previous = self.generator_losses.copy()
            # TBD: add epoch bigger than history length
#             if len(self.history["generator_loss"]) == epoch:
#                 self.history["generator_loss"].append(
#                     np.mean(self.generator_losses))
#                 self.history["f1_score_train"].append(
#                     np.mean(self.train_f1_score_list))
#                 self.history["f1_score_valid"].append(
#                     np.mean(self.valid_f1_score_list))
#             elif len(self.history["generator_loss"]) < epoch:
#                 self.history["generator_loss"][epoch] = np.mean(
#                     self.generator_losses)
#                 self.history["f1_score_train"][epoch] = np.mean(
#                     train_f1_score_list)
#                 self.history["f1_score_valid"][epoch] = np.mean(
#                     valid_f1_score_list)
    def learning_rate_scheduler(self, learning_rate, epoch):
        # poly_lr
        exponent = 0.9
        max_epoch = 575
        new_learning_rate = learning_rate * (1 - epoch / max_epoch)**exponent

        return new_learning_rate

    def get_info_folderPath(self):
        return (
            str(round(self.generator_loss_min, 5))
            + "_"
            + str(round(self.generator_loss_max_min, 5))
        )

    def save_study_info(self, path=None):

        if path == None:
            path = self.temp_weights_path

        generator_weigth_path = os.path.join(path, "generator.h5")
        discriminator_weigth_path = os.path.join(path, "discriminator.h5")
        combined_weigth_path = os.path.join(path, "combined.h5")

        self.generator.save_weights(generator_weigth_path)
        self.discriminator.save_weights(discriminator_weigth_path)
        self.combined.save_weights(combined_weigth_path)

        study_info = {}
        study_info["start_epoch"] = self.start_epoch
        study_info["generator_loss_min"] = self.generator_loss_min
        study_info["generator_loss_max_min"] = self.generator_loss_max_min
        study_info["generator_loss_min_min"] = self.generator_loss_min_min
        study_info["generator_losses_previous"] = self.generator_losses_previous
        study_info["discriminator_acces"] = self.discriminator_acces
        study_info["history"] = self.history
        study_info["train_loaded_data_index"] = self.train_loaded_data_index
        file = open(path + "/study_info.pkl", "wb")
        dump(study_info, file)
        file.close()

    def load_study_info(self):

        self.generator.load_weights("generator.h5")
        self.discriminator.load_weights("discriminator.h5")
        self.combined.load_weights("combined.h5")

        if os.path.isfile("study_info.pkl"):
            file = open("study_info.pkl", "rb")
            study_info = load(file)
            file.close()
            self.start_epoch = study_info["start_epoch"]
            self.generator_loss_min = study_info["generator_loss_min"]
            self.generator_loss_max_min = study_info["generator_loss_max_min"]
            self.generator_loss_min_min = study_info["generator_loss_min_min"]
            self.generator_losses_previous = study_info["generator_losses_previous"]
            self.discriminator_acces = study_info["discriminator_acces"]
            self.history = study_info["history"]
            self.train_loaded_data_index = study_info["train_loaded_data_index"] 
        else:
            print("No info pkl file!")

    def load_best_weights(self):
        self.generator.load_weights(self.temp_weights_path + "/generator.h5")
        self.discriminator.load_weights(
            self.temp_weights_path + "/discriminator.h5")
        self.combined.load_weights(self.temp_weights_path + "/combined.h5")
            

In [5]:
generator_lr = 1e-3
discriminator_lr = 1e-4
batch_size = 10
g_lr = generator_lr * batch_size
d_lr = discriminator_lr * batch_size
gan = Pix2PixSegmentation(generator_power=4, discriminator_power=4, generator_learning_rate=g_lr, discriminator_learning_rate=d_lr,
                          test=False)

In [6]:
#gan.find_error = True
#gan.find_error_epoch = 5
#gan.load_study_info()
#gan.load_study_info()
gan.start_epoch = 0
gan.train(epochs=575, batch_size=batch_size, sample_interval=3100)

current_generator_loss_median : 1.0
discriminator_learning is True
[Epoch 0/575] [Batch 0/6200] [D loss: 0.187494, acc:  26%] [G loss: 32.489243] time: 0:00:14.140961


[Epoch 0/575] [Batch 3100/6200] [D loss: 0.123670, acc:  41%] [G loss: 18.639917] time: 0:04:10.046226


discriminator_acces : 0.3739344928916844
Mean generator_loss : 19.722179412841797
Max generator_loss : 37.53890609741211
Min generator_loss : 15.17371654510498
generator loss decrease : 80.2778205871582
generator loss decrease ratio : (0.19722179412841798)
Max generator loss decrease : 962.4610939025879
current lowest generator loss : 100
current Learning_rate = 0.01
save weights
train_f1_loss : 13.07392406463623
train_f1_score : 0.1828797459602356
train_f1_rounded_score : 0.2582857608795166


valid_f1_loss : 13.432496070861816
valid_f1_score : 0.16046899557113647
valid_f1_rounded_score : 0.26145774126052856
current_generator_loss_median : 18.49822425842285
discriminator_learning is True
[Epoch 1/575] [Batch 0/6200] [D loss: 0.120977, acc:  58%] [G loss: 18.030672] time: 0:10:40.839913


[Epoch 1/575] [Batch 3100/6200] [D loss: 0.152301, acc:   8%] [G loss: 15.264763] time: 0:14:32.750396


discriminator_acces : 0.25406423224707425
Mean generator_loss : 16.779293060302734
Max generator_loss : 22.005199432373047
Min generator_loss : 13.718571662902832
generator loss decrease : 2.9428863525390625
generator loss decrease ratio : (0.8507829308509827)
Max generator loss decrease : 15.533706665039062
current lowest generator loss : 19.722179412841797
current Learning_rate = 0.009984346464159641
save weights
train_f1_loss : 12.895927429199219
train_f1_score : 0.19400453567504883
train_f1_rounded_score : 0.323555588722229


valid_f1_loss : 13.062210083007812
valid_f1_score : 0.18361186981201172
valid_f1_rounded_score : 0.4198073744773865
current_generator_loss_median : 15.823863983154297
discriminator_learning is True
[Epoch 2/575] [Batch 0/6200] [D loss: 0.203336, acc:   5%] [G loss: 15.847067] time: 0:21:05.711703


ERROR:root:Internal Python error in the inspect module.                       |
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gr300\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-73abfbf0db1f>", line 6, in <module>
    gan.train(epochs=575, batch_size=batch_size, sample_interval=3100)
  File "<ipython-input-4-769d48d7de45>", line 335, in train
    generator_loss = self.combined.train_on_batch(
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1698, in train_on_batch
    self.reset_metrics()
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1637, in reset_metrics
    m.reset_states()
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\metrics.py", line 247, in reset_states
    K.batch_set_value([(v, 0) for v in self.variables])
  File "C:\Users\gr300\AppData\Roaming\P

TypeError: object of type 'NoneType' has no len()

In [3]:
#gan.find_error = True
#gan.find_error_epoch = 5
#gan.load_study_info()
#gan.load_study_info()
gan.start_epoch = 0
gan.train(epochs=575, batch_size=batch_size, sample_interval=3100)

current_generator_loss_median : 1.0
discriminator_learning is True
[Epoch 0/575] [Batch 0/6200] [D loss: 0.192542, acc:  34%] [G loss: 35.593498] time: 0:00:15.636787


[Epoch 0/575] [Batch 3100/6200] [D loss: 0.127238, acc:  95%] [G loss: 19.044613] time: 0:04:11.519256


discriminator_acces : 0.8540087398622305
Mean generator_loss : 19.739973068237305
Max generator_loss : 35.59349822998047
Min generator_loss : 14.8356294631958
generator loss decrease : 80.2600269317627
generator loss decrease ratio : (0.19739973068237304)
Max generator loss decrease : 964.4065017700195
current lowest generator loss : 100
current Learning_rate = 0.01
save weights
train_f1_loss : 12.714962005615234
train_f1_score : 0.20531487464904785
train_f1_rounded_score : 0.4767947196960449


valid_f1_loss : 13.072277069091797
valid_f1_score : 0.1829826831817627
valid_f1_rounded_score : 0.5394037961959839
current_generator_loss_median : 18.517194747924805
discriminator_learning is False
[Epoch 1/575] [Batch 0/6200] [D loss: 0.127271, acc:  97%] [G loss: 18.393978] time: 0:10:43.043007


[Epoch 1/575] [Batch 3100/6200] [D loss: 0.128559, acc:  60%] [G loss: 16.446007] time: 0:13:54.043540


...

discriminator_acces : 0.9609368801116943
Mean generator_loss : 8.927451133728027
Max generator_loss : 12.605867385864258
Min generator_loss : 6.539391994476318
generator loss decrease : 0.015118598937988281
generator loss decrease ratio : (0.9983093738555908)
Max generator loss decrease : 0.8217391967773438
current lowest generator loss : 8.942569732666016
current Learning_rate = 0.008960762028300732
save weights
train_f1_loss : 9.680624961853027
train_f1_score : 0.3949609398841858
train_f1_rounded_score : 0.36503690481185913


valid_f1_loss : 9.787391662597656
valid_f1_score : 0.3882880210876465
valid_f1_rounded_score : 0.38218826055526733
current_generator_loss_median : 8.166941404342651
discriminator_learning is False
[Epoch 67/575] [Batch 0/6200] [D loss: 0.071172, acc:  96%] [G loss: 9.878385] time: 13:13:05.672289


ERROR:root:Internal Python error in the inspect module.                       |
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gr300\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-73abfbf0db1f>", line 6, in <module>
    gan.train(epochs=575, batch_size=batch_size, sample_interval=3100)
  File "<ipython-input-1-faf2df80b3d8>", line 331, in train
    generator_loss = self.combined.train_on_batch(
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1691, in train_on_batch
    iterator = data_adapter.single_batch_iterator(self.distribute_strategy, x,
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 1534, in single_batch_iterator
    return iter(dataset)
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 415, in __iter__
    return iterator_ops

TypeError: object of type 'NoneType' has no len()

In [3]:
#gan.find_error = True
#gan.find_error_epoch = 5
#gan.load_study_info()
gan.load_study_info()
gan.start_epoch = 5
gan.train(epochs=575, batch_size=batch_size, sample_interval=3100)

current_generator_loss_median : 13.20044493675232
discriminator_learning is True
[Epoch 5/575] [Batch 0/6200] [D loss: 0.123203, acc:  88%] [G loss: 16.688765] time: 0:00:17.875881


[Epoch 5/575] [Batch 3100/6200] [D loss: 0.135997, acc:  22%] [G loss: 12.500026] time: 0:04:14.702376


discriminator_acces : 0.6007949733686062
Mean generator_loss : 13.601099967956543
Max generator_loss : 22.40525245666504
Min generator_loss : 9.019099235534668
generator loss decrease : -0.06017589569091797
generator loss decrease ratio : (1.0044440031051636)
Max generator loss decrease : 977.594747543335
current lowest generator loss : 13.540924072265625
current Learning_rate = 0.00992170499498161
train_f1_loss : 12.09400463104248
train_f1_score : 0.24412471055984497
train_f1_rounded_score : 0.4313807189464569


valid_f1_loss : 12.781795501708984
valid_f1_score : 0.20113778114318848
valid_f1_rounded_score : 0.36950501799583435
current_generator_loss_median : 11.76853895187378
discriminator_learning is True
[Epoch 6/575] [Batch 0/6200] [D loss: 0.160582, acc:   0%] [G loss: 16.573912] time: 0:11:59.825378


[Epoch 6/575] [Batch 3100/6200] [D loss: 0.123661, acc:  62%] [G loss: 12.319873] time: 0:17:47.614875


...

valid_f1_loss : 10.55346965789795
valid_f1_score : 0.3404081463813782
valid_f1_rounded_score : 0.5519055724143982
current_generator_loss_median : 11.584134101867676
discriminator_learning is False
[Epoch 11/575] [Batch 0/6200] [D loss: 0.122437, acc:  92%] [G loss: 11.558370] time: 1:18:07.183506


[Epoch 11/575] [Batch 3100/6200] [D loss: 0.142673, acc:   1%] [G loss: 12.604108] time: 1:22:52.716507


discriminator_acces : 0.08811011518442727
Mean generator_loss : 12.27579116821289
Max generator_loss : 17.51087188720703
Min generator_loss : 9.178255081176758
generator loss decrease : 0.3339414596557617
generator loss decrease ratio : (0.9735171794891357)
Max generator loss decrease : 0.8266792297363281
current lowest generator loss : 12.609732627868652
current Learning_rate = 0.009827660231920667
save weights
train_f1_loss : 10.53527545928955
train_f1_score : 0.3415452837944031
train_f1_rounded_score : 0.504142701625824


valid_f1_loss : 10.259178161621094
valid_f1_score : 0.35880136489868164
valid_f1_rounded_score : 0.5682680606842041
current_generator_loss_median : 11.30786657333374
discriminator_learning is True
[Epoch 12/575] [Batch 0/6200] [D loss: 0.139826, acc:   6%] [G loss: 11.350702] time: 1:30:00.138506


KeyboardInterrupt: 